In [29]:
import scrapy 
from scrapy import Spider, Request
import re
import pandas as pd
import json
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings



zips = pd.read_csv('smh_zips.txt')
zip_codes = zips['ZipCode']
zip_codes =zip_codes.tolist()
    
class SchoolSpider(scrapy.Spider):

    name = "school_scraper"

    start_urls = [f'https://www.greatschools.org/search/search.zipcode?page=1&sort=rating&zip=20852']

    def parse(self, response):
        res = response.xpath("//script").extract_first()
        x = json.loads(re.search(r'gon.search=(.*?);', res).group(1))
        num_pages = x['totalPages'] 
        num_pages += 1

        page_urls = ['https://www.greatschools.org/search/search.zipcode?page=' + str(i) + '&sort=rating&zip=' + str(z) for i in range(1, num_pages) for z in zip_codes]

        print('pageurls =', page_urls)

        for url in page_urls:
            yield Request(url = url, callback = self.parse_results)

    def parse_results(self, response):

        res_2 = response.xpath("//script").extract_first()
        y = json.loads(re.search(r'gon.search=(.*?);', res_2).group(1))
        schools = y['schools']

        dict_list = []

        dict = {}
        for i in range(0, len(schools)):
            temp = schools[i]
            dict['name'] = temp['name']
            dict['street'] = temp['address']['street1']
            dict['city'] = temp['address']['city']
            dict['zip'] = temp['address']['zip']
            dict['state'] = temp['state']
            dict['lat'] = temp['lat']
            dict['long'] = temp['lon']
            dict['districtID'] = temp['districtId']
            dict['districtName'] = temp['districtName']
            dict['gradeLevels'] = temp['gradeLevels']
            dict['ratingScale'] =  temp['ratingScale']
            dict['enrollment'] = temp['enrollment']
            dict['schoolType'] = temp['schoolType']
            dict['pctLowIncome']= temp['percentLowIncome']
            dict['overallRating']  = temp['rating']
            dict_list.append(dict)
            yield dict

In [35]:

process = CrawlerProcess({
    'FEED_FORMAT': 'CSV', 
    'FEED_URI': 'great_schools_scraped.csv',
})

# 'followall' is the name of one of the spiders of the project.
process.crawl(SchoolSpider)
process.start() # the script will block here until the crawling is finished

2021-12-09 14:36:16 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-12-09 14:36:16 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 21.2.0, Python 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.4.7, Platform Windows-10-10.0.17134-SP0
2021-12-09 14:36:16 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-12-09 14:36:16 [scrapy.crawler] INFO: Overridden settings:
{}
2021-12-09 14:36:16 [scrapy.extensions.telnet] INFO: Telnet Password: c70e27f6bf7fb8a6
2021-12-09 14:36:16 [scrapy.extensions.feedexport] ERROR: Unknown feed format: CSV
2021-12-09 14:36:16 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-12-09 14:36:16 [scrapy.middleware] INFO: Enabled downl

ReactorNotRestartable: 